In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import statistics as stat

In [2]:
def allreduce_scaleup_intel_v2(z, a, n_ranks_per_node, b_xe, b_mdfi, tile_to_tile=False, gpu_to_gpu=False):
    """
    For the scale-up case the Allreduce is done in a Reduce-Scatter and allgather phases.
    We have contribution from both the MDFI (title-to-tile, within GPU) and Xe-link (GPU-to-GPU)
    data movement contibution here. Each phase is a combination of the tile-to-tile and 
    GPU-to-GPU data movement,therefore a factor of 2 is needed for capturing both contibutions
    correctly
    a: Inital data chunck, oneCCL divides the initial buffer in half to distribute parallely between two tiles, default=2
    """
    assert not n_ranks_per_node == 1
    n_xe = n_mdfi = n_ranks_per_node // 2
    #print(n_xe, n_mdfi)
    if n_ranks_per_node == 2 and tile_to_tile is True:
        assert gpu_to_gpu is False
        return (2 * (z/a)/n_mdfi/b_mdfi) * 1000
    if n_ranks_per_node == 2 and gpu_to_gpu is True:
        assert tile_to_tile is False
        return (2 * (z/a)/n_xe/b_xe) * 1000
    return ((2 * (z/a)/n_xe/b_xe) + (2 * (z/a)/n_mdfi/b_mdfi)) * 1000

In [3]:
t_AR_N1R2_t2t = allreduce_scaleup_intel_v2(z=0.875, a=2, n_ranks_per_node=2, b_xe=15, 
                                           b_mdfi=180, tile_to_tile=True, gpu_to_gpu=False)
print(f"Allreduce time 0.875 GB tile-to-tile: N1xR2 = {t_AR_N1R2_t2t} ms")

t_AR_N1R2_g2g = allreduce_scaleup_intel_v2(z=0.875, a=2, n_ranks_per_node=2, b_xe=15, 
                                           b_mdfi=180, tile_to_tile=False, gpu_to_gpu=True)
print(f"Allreduce time 0.875 GB gpu-to-gpu: N1xR2 = {t_AR_N1R2_g2g} ms")

t_AR_N1R4 = allreduce_scaleup_intel_v2(z=0.875, a=2, n_ranks_per_node=4, b_xe=15, 
                                           b_mdfi=180, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 0.875 GB: N1xR4 = {t_AR_N1R4} ms")

t_AR_N1R8 = allreduce_scaleup_intel_v2(z=0.875, a=2, n_ranks_per_node=8, b_xe=15, 
                                           b_mdfi=180, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 0.875 GB: N1xR8 = {t_AR_N1R8} ms")

t_AR_N1R12 = allreduce_scaleup_intel_v2(z=0.875, a=2, n_ranks_per_node=12, b_xe=15, 
                                           b_mdfi=180, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 0.875 GB: N1xR12 = {t_AR_N1R12} ms")

Allreduce time 0.875 GB tile-to-tile: N1xR2 = 4.861111111111112 ms
Allreduce time 0.875 GB gpu-to-gpu: N1xR2 = 58.333333333333336 ms
Allreduce time 0.875 GB: N1xR4 = 31.59722222222222 ms
Allreduce time 0.875 GB: N1xR8 = 15.79861111111111 ms
Allreduce time 0.875 GB: N1xR12 = 10.532407407407407 ms


In [4]:
def allreduce_scaleout_intel_v2(z, a, n_ranks_per_node, n_nodes, n_nic_per_node, b_xe, b_mdfi, 
                                b_nic, tile_to_tile=False, gpu_to_gpu=False):
    tot_b_nic = 0.8 * (n_nic_per_node * b_nic)
    if n_ranks_per_node == 1:
        time = ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic) * 1000
    elif n_ranks_per_node == 2:
        if tile_to_tile:
            assert gpu_to_gpu is False
            n_mdfi = n_ranks_per_node // 2
            time = ((2 * ((z/a)/n_mdfi/b_mdfi)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)) * 1000
        if gpu_to_gpu:
            assert tile_to_tile is False
            n_xe = n_ranks_per_node // 2
            time = ((2 * ((z/a)/n_xe/b_xe)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)) * 1000
    else:
        assert n_ranks_per_node % 2 == 0
        n_xe = n_ranks_per_node // 2
        n_mdfi = n_ranks_per_node // 2
        time = ((2 * ((z/a)/n_xe/b_xe)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)) * 1000
    return time

In [13]:
B_NIC=23

t_AR_N2R1 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=1, n_nodes=2, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N2xR1 = {t_AR_N2R1} ms")

t_AR_N4R1 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=1, n_nodes=4, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N4xR1 = {t_AR_N4R1} ms")

t_AR_N8R1 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=1, n_nodes=8, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N8xR1 = {t_AR_N8R1} ms")

t_AR_N16R1 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=1, n_nodes=16, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N16xR1 = {t_AR_N16R1} ms")
##
t_AR_N2R2_t2t = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=2, n_nodes=2, n_nic_per_node=2, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=True, gpu_to_gpu=False)
print(f"\nAllreduce time 2 GB: N2xR2, tile-to-tile = {t_AR_N2R2_t2t} ms")
t_AR_N2R4 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=4, n_nodes=2, n_nic_per_node=4, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N2xR4 = {t_AR_N2R4} ms")

t_AR_N2R6 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=6, n_nodes=2, n_nic_per_node=6, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N2xR6 = {t_AR_N2R6} ms")

t_AR_N2R8 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=8, n_nodes=2, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N2xR8 = {t_AR_N2R8} ms")

t_AR_N2R12 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=12, n_nodes=2, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N2xR12 = {t_AR_N2R12} ms")
##
t_AR_N4R12 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=12, n_nodes=4, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"\nAllreduce time 2 GB: N4xR12 = {t_AR_N4R12} ms")
t_AR_N8R12 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=12, n_nodes=8, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N8xR12 = {t_AR_N8R12} ms")
t_AR_N16R12 = allreduce_scaleout_intel_v2(z=2, a=2, n_ranks_per_node=12, n_nodes=16, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB: N16xR12 = {t_AR_N16R12} ms")


print(f"\nSpecial Case")
t_AR_N256R12 = allreduce_scaleout_intel_v2(z=1, a=2, n_ranks_per_node=12, n_nodes=256, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=B_NIC, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 1 GB: N256xR12 = {t_AR_N256R12} ms")

Allreduce time 2 GB: N2xR1 = 108.69565217391303 ms
Allreduce time 2 GB: N4xR1 = 163.04347826086953 ms
Allreduce time 2 GB: N8xR1 = 190.2173913043478 ms
Allreduce time 2 GB: N16xR1 = 203.80434782608694 ms

Allreduce time 2 GB: N2xR2, tile-to-tile = 65.45893719806763 ms
Allreduce time 2 GB: N2xR4 = 93.84057971014492 ms
Allreduce time 2 GB: N2xR6 = 62.56038647342994 ms
Allreduce time 2 GB: N2xR8 = 46.92028985507246 ms
Allreduce time 2 GB: N2xR12 = 35.80917874396135 ms

Allreduce time 2 GB: N4xR12 = 42.60265700483092 ms
Allreduce time 2 GB: N8xR12 = 45.9993961352657 ms
Allreduce time 2 GB: N16xR12 = 47.69776570048309 ms

Special Case
Allreduce time 1 GB: N256xR12 = 24.644993583937193 ms


In [6]:
((2 * (2 * (2 -1) / 2)) / 23) * 1000

86.95652173913044

In [15]:
def allreduce_scaleout_intel_pcie(z, a, n_ranks_per_node, n_nodes, n_nic_per_node, b_xe, b_mdfi, 
                                b_nic, b_pcie, f_pcie, tile_to_tile=False, gpu_to_gpu=False):
    """
    f_pcie: A factor determining how many times PCIe has been used in the data movement. 
    Naively, without any staging, pipelining etc. the PCIe gets used 3 times, GPU --> DDR,
    DDR --> Fabric, and again DDR --> GPU. Default: 1, used just once.
    """
    tot_b_nic = 0.8 * (n_nic_per_node * b_nic)
    if n_ranks_per_node == 1:
        time = (((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic) + (2 * z * f_pcie / b_pcie)) * 1000
    elif n_ranks_per_node == 2:
        if tile_to_tile:
            assert gpu_to_gpu is False
            n_mdfi = n_ranks_per_node // 2
            time = ((2 * ((z/a)/n_mdfi/b_mdfi)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)
                   + (2 * z * f_pcie / n_ranks_per_node / b_pcie)) * 1000
        if gpu_to_gpu:
            assert tile_to_tile is False
            n_xe = n_ranks_per_node // 2
            time = ((2 * ((z/a)/n_xe/b_xe)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)
                   + (2 * z * f_pcie / n_ranks_per_node / b_pcie)) * 1000
    else:
        assert n_ranks_per_node % 2 == 0
        n_xe = n_ranks_per_node // 2
        n_mdfi = n_ranks_per_node // 2
        time = ((2 * ((z/a)/n_xe/b_xe)) + ((2 * (z * (n_nodes -1) / n_nodes)) / tot_b_nic)
               + (2 * z * f_pcie / n_ranks_per_node / b_pcie)) * 1000
    return time

In [16]:
F_PCIE=2

t_AR_N2R1_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=1, n_nodes=2, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N2xR1 = {t_AR_N2R1_pcie} ms")

t_AR_N4R1_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=1, n_nodes=4, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N4xR1 = {t_AR_N4R1_pcie} ms")

t_AR_N8R1_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=1, n_nodes=8, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N8xR1 = {t_AR_N8R1_pcie} ms")

t_AR_N16R1_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=1, n_nodes=16, n_nic_per_node=1, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N16xR1 = {t_AR_N16R1_pcie} ms")
##

t_AR_N2R2_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=2, n_nodes=2, n_nic_per_node=2, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=True, gpu_to_gpu=False)
print(f"\nAllreduce time 2 GB PCIe: N2xR2, tile-to-tile = {t_AR_N2R2_pcie} ms")

t_AR_N2R4_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=4, n_nodes=2, n_nic_per_node=4, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N2xR4 = {t_AR_N2R4_pcie} ms")
t_AR_N2R6_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=6, n_nodes=2, n_nic_per_node=6, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N2xR6 = {t_AR_N2R6_pcie} ms")
t_AR_N2R8_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=8, n_nodes=2, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N2xR8 = {t_AR_N2R8_pcie} ms")


t_AR_N2R12_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=12, n_nodes=2, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"\nAllreduce time 2 GB PCIe: N2xR12 = {t_AR_N2R12_pcie} ms")
t_AR_N4R12_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=12, n_nodes=4, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N4xR12 = {t_AR_N4R12_pcie} ms")
t_AR_N8R12_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=12, n_nodes=8, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N8xR12 = {t_AR_N8R12_pcie} ms")
t_AR_N16R12_pcie = allreduce_scaleout_intel_pcie(z=2, a=2, n_ranks_per_node=12, n_nodes=16, n_nic_per_node=8, b_xe=15, 
                                         b_mdfi=180, b_nic=23, b_pcie=42, f_pcie=F_PCIE, tile_to_tile=False, gpu_to_gpu=False)
print(f"Allreduce time 2 GB PCIe: N16xR12 = {t_AR_N16R12_pcie} ms")

Allreduce time 2 GB PCIe: N2xR1 = 299.1718426501035 ms
Allreduce time 2 GB PCIe: N4xR1 = 353.51966873706 ms
Allreduce time 2 GB PCIe: N8xR1 = 380.69358178053824 ms
Allreduce time 2 GB PCIe: N16xR1 = 394.28053830227736 ms

Allreduce time 2 GB PCIe: N2xR2, tile-to-tile = 160.69703243616286 ms
Allreduce time 2 GB PCIe: N2xR4 = 141.45962732919253 ms
Allreduce time 2 GB PCIe: N2xR6 = 94.30641821946169 ms
Allreduce time 2 GB PCIe: N2xR8 = 70.72981366459626 ms

Allreduce time 2 GB PCIe: N2xR12 = 51.68219461697722 ms
Allreduce time 2 GB PCIe: N4xR12 = 58.47567287784679 ms
Allreduce time 2 GB PCIe: N8xR12 = 61.87241200828157 ms
Allreduce time 2 GB PCIe: N16xR12 = 63.57078157349896 ms


### Experimental Data with oneCCL 2 GB Buffer size

In [19]:
exp_AR_N2R1 = np.array([423.5, 424.2, 425.4, 402.8, 380.8, 403.3, 380.9, 380.9])
exp_AR_N2R2 = np.array([522.0, 468.3, 427.2, 407.1, 430.6, 431.6, 422.9, 426.4])
exp_AR_N2R4 = np.array([472.6, 503.8, 507.2, 505.1, 477.6, 486.3, 495.3, 506.1])
exp_AR_N2R6 = np.array([368.0, 358.0, 341.2, 349.0, 350.3, 364.4, 346.8, 348.0])
exp_AR_N2R8 = np.array([204.0, 178.3, 205.7, 200.4, 204.7, 208.0, 211.6, 212.9])
exp_AR_N2R12 = np.array([225.6, 217.4, 217.1, 224.1, 222.7, 225.2, 222.1, 220.4])
##
exp_AR_N4R12 = np.array([238.5, 235.7, 206.5, 237.0, 235.6, 236.9, 237.5, 236.0])
exp_AR_N8R12 = np.array([247.1, 248.0, 247.4, 248.9, 249.8, 248.5, 247.8, 250.8])
exp_AR_N16R12 = np.array([251.5, 245.6, 247.3, 251.1, 246.3, 243.9, 251.1, 246.1])
##
exp_AR_N4R1 = np.array([584.7, 555.7, 566.3, 559.1, 559.5, 559.8, 528.9, 528.9])
exp_AR_N8R1 = np.array([643.0, 669.5, 659.5, 610.5, 627.4, 638.5, 619.0, 625.5])
exp_AR_N16R1 = np.array([687.7, 680.0,  675.3, 681.4, 674.9, 681.3, 683.9, 680.5])

t_avg_AR_N2R1 = np.mean(exp_AR_N2R1)
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR1 = {np.mean(exp_AR_N2R1)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR2 = {np.mean(exp_AR_N2R2)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR4 = {np.mean(exp_AR_N2R4)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR6 = {np.mean(exp_AR_N2R6)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR8 = {np.mean(exp_AR_N2R8)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR12 = {np.mean(exp_AR_N2R12)} ms")
##
print(f"\nMeasured Allreduce time 2 GB (avg. over 8 iterations), N4xR12 = {np.mean(exp_AR_N4R12)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N8xR12 = {np.mean(exp_AR_N8R12)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N16xR12 = {np.mean(exp_AR_N16R12)} ms")
##
print(f"\nMeasured Allreduce time 2 GB (avg. over 8 iterations), N2xR1 = {np.mean(exp_AR_N2R1)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N4xR1 = {np.mean(exp_AR_N4R1)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N8xR1 = {np.mean(exp_AR_N8R1)} ms")
print(f"Measured Allreduce time 2 GB (avg. over 8 iterations), N16xR1 = {np.mean(exp_AR_N16R1)} ms \n")

exp_AR875_N1R2_t2t = np.array([6.886161, 6.999721, 7.077407, 6.967706, 6.741716, 7.204491, 7.423761]) ## ZE01
exp_AR875_N1R2 = np.array([73.846469,  73.698414, 74.28551, 76.365497, 73.650447, 73.741025, 74.074025]) ## ZE03
exp_AR875_N1R4 = np.array([43.095799,   42.898535, 41.199356, 40.680996, 39.770582, 41.755462, 41.19378])
exp_AR875_N1R12 = np.array([18.309431,    19.994012,    20.699726, 19.172793, 20.112963, 19.920254, 19.827353])
print(f"Measured Allreduce time 0.875 GB (avg. over 7 iterations) tile-to-tile, N1xR2 = {np.mean(exp_AR875_N1R2_t2t)} ms")
print(f"Measured Allreduce time 0.875 GB (avg. over 7 iterations) gpu-to-gpu, N1xR2 = {np.mean(exp_AR875_N1R2)} ms")
print(f"Measured Allreduce time 0.875 GB (avg. over 7 iterations), N1xR4 = {np.mean(exp_AR875_N1R4)} ms")
print(f"Measured Allreduce time 0.875 GB (avg. over 7 iterations), N1xR12 = {np.mean(exp_AR875_N1R12)} ms")

Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR1 = 402.725 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR2 = 442.0125 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR4 = 494.25 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR6 = 353.2125 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR8 = 203.20000000000002 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR12 = 221.825 ms

Measured Allreduce time 2 GB (avg. over 8 iterations), N4xR12 = 232.9625 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N8xR12 = 248.53750000000002 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N16xR12 = 247.8625 ms

Measured Allreduce time 2 GB (avg. over 8 iterations), N2xR1 = 402.725 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N4xR1 = 555.3625 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N8xR1 = 636.6125 ms
Measured Allreduce time 2 GB (avg. over 8 iterations), N16xR1 = 680.625 ms 

M